In [ ]:
from IPython.display import display
from ipywidgets import widgets
import datetime
import math
import os
import csv
import numpy as np
import statistics
from vega import VegaLite

# see https://github.com/vega/vega-lite/

rootdir = "d:/temp/performance/new_benchmark2"

width = 600
height = 332

class Report(object):
    def __init__(self, d):
        self.__dict__ = d
        
class Result():
    def __init__(self):
        self.Name = ""
        self.Index = 0
        self.TimeMean = 0
        self.TimeMax = 0
        self.TimeMin = 0
        self.TimeStdDev = 0
        self.TimeSlope = 0
        self.MemoryMean = 0
        self.MemoryStdDev = 0
        self.MemorySlope = 0
        self.CpuMean = 0
        self.CpuStdDev = 0
        self.CpuSlope = 0

class Benchmark():
    def __init__(self, filename, data):
        self.filename = filename
        self.data = data
        
def strip_dict(d):
    h = {}
    for k in d:
        ks = k.replace(" ","").replace("\ufeff","")
        h[ks] = d[k].strip()
    return h

def load_report(filename):
    result = []
    with open(filename, "r", encoding='utf8') as f:
        dr = csv.DictReader(f);
        for row in dr:
            row = strip_dict(row)
            r = Report(row)
            r.TimeMean = float(r.TimeMean)
            r.TimeStdDev = float(r.TimeStdDev)
            r.TimeMax = r.TimeMean + r.TimeStdDev
            r.TimeMin = r.TimeMean - r.TimeStdDev
            r.TimeSlope = float(r.TimeSlope)
            r.MemoryMean = float(r.MemoryMean)
            r.MemoryStdDev = float(r.MemoryStdDev)
            r.MemorySlope = float(r.MemorySlope)
            r.CpuMean = float(r.CpuMean)
            r.CpuStdDev = float(r.CpuStdDev)
            r.CpuSlope = float(r.CpuSlope)
        
            result += [r]
    return result
                
def load_benchmarks(rootdir):
    benchmarks = []
    for name in os.listdir(rootdir):
        dir = os.path.join(rootdir, name)
        if name.startswith("benchmark_") and not name.endswith(".zip"):
            for report in os.listdir(dir):
                if report.endswith("summary.csv"):
                    filename = os.path.join(rootdir, name, report)
                    r = load_report(filename)
                    benchmarks += [Benchmark(filename, r)]
    return benchmarks

benchmarks = load_benchmarks(rootdir)
tests = [i.Test for i in benchmarks[0].data]
graphs = []

for t in tests:
    data = []
    index = 0
    for b in benchmarks:
        for row in b.data:
            if row.Test == t:
                row.Index = index
                index += 1
                data += [row.__dict__]
    graphs += [(t, data)]
    
def vega_spec(dataset):
    global width, height
    h = len(dataset)
    spec = {
        "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
        "selection": {
          "grid": { "type": "interval", "bind": "scales" }            
        },
        "columns": 1,
        "concat": [            
        ]
    }
    for name, data in dataset:
        spec["concat"] += [{
            "title": name,
            "width": width, "height": height,
            "data": {"values": data},
            "layer":[                
                {
                    "mark": {
                        "type": "errorband",
                        "extent": "ci"
                    },
                    "encoding": {
                        "x": {"field": "Index", "type": "quantitative", "scale": {"padding": 0, "zero": False}},
                        "y": {"field": "TimeMin", "type": "quantitative", "scale": {"padding": 0, "zero": False}},
                        "y2": {"field": "TimeMax", "type": "quantitative", "scale": {"padding": 0, "zero": False}}
                    }
                },
                {
                    "mark": "line",
                    "encoding": {
                        "x": {"field": "Index", "type": "quantitative", "scale": {"padding": 0, "zero": False}},
                        "y": {"field": "TimeMean", "type": "quantitative", "scale": {"padding": 0, "zero": False}},
                    }
                }
            ]
        }]
    return spec

VegaLite(vega_spec(graphs))